In [1]:
# Compilation and Accounting Data
from bs4 import BeautifulSoup
import requests
import os
import wget
import time
base_path = "C:\\Users\\10235555\\Documents\\Dataportal Data\\Compilation and Accounting Data"
import urllib.request
import logging
from selenium.webdriver.support.ui import Select
import datetime
from Hashing.HashScrapedData import _hashing
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import ElementNotInteractableException, NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
import time
import os
from RequestInferSchemaToJsonAPI.main import TriggerInferShemaToJsonAPI

In [2]:
chrome_options = webdriver.ChromeOptions()
base_path = "C:\\Users\\10235555\\Documents\\Dataportal Data\\Compilation and Accounting Data" # local, gets current working directory
prefs = {'download.default_directory' : base_path}#, "profile.content_settings.exceptions.automatic_downloads.*.setting" : 1}
chrome_options.add_experimental_option('prefs', prefs)
# chrome_options.add_argument('--user-agent="Mozilla/5.0 (Windows Phone 10.0; Android 4.2.1; Microsoft; Lumia 640 XL LTE) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Mobile Safari/537.36 Edge/12.10166"')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
driver.maximize_window()

In [22]:
url_1 = 'https://di.unfccc.int/flex_cad'
url_2 = 'https://di.unfccc.int/flex_cad2'
url_list = [url_1,url_2]
for url in url_list:
    driver.get(url)
    time.sleep(3)
    content = driver.find_element(By.CLASS_NAME,'detailed-party-view')
    topic = content.find_element(By.TAG_NAME,'h1').text
    tit = topic.split(' - ')[1]
    search_block = driver.find_element(By.CLASS_NAME,'search-block').find_elements(By.CLASS_NAME,'column')
    # party
    party = search_block[0]
    driver.execute_script("arguments[0].style.display = 'block';", party)
    time.sleep(1)
    filter_box_1 = party.find_element(By.CLASS_NAME,'ddlbFilterBox')
    driver.execute_script("arguments[0].style.display = 'block';", filter_box_1)
    Select_all_part = filter_box_1.find_element(By.CLASS_NAME,"ddlbButton")
    Select_all_part.click()
    # year 
    year = search_block[2]
    driver.execute_script("arguments[0].style.display = 'block';", year)
    time.sleep(1)
    filter_box_2 = year.find_element(By.CLASS_NAME,'ddlbFilterBox')
    driver.execute_script("arguments[0].style.display = 'block';", filter_box_2)
    time.sleep(1)
    Select_all_year = filter_box_2.find_element(By.CLASS_NAME,"ddlbButton")
    Select_all_year.click()
    # category
    category = search_block[3]
    driver.execute_script("arguments[0].style.display = 'block';", category)
    time.sleep(1)
    filter_box_3 = category.find_element(By.CLASS_NAME,'ddlbFilterBox')
    driver.execute_script("arguments[0].style.display = 'block';", filter_box_3)
    time.sleep(1)
    list_category = driver.find_element(By.ID,'listboxdiv_categories').find_elements(By.TAG_NAME,'p')
    for categ in list_category:
        categ.click()
        time.sleep(1)
        # Classification
        Classification = search_block[4]
        driver.execute_script("arguments[0].style.display = 'block';", Classification)
        time.sleep(1)
        filter_box_4 = Classification.find_element(By.CLASS_NAME,'ddlbFilterBox')
        driver.execute_script("arguments[0].style.display = 'block';", filter_box_4)
        time.sleep(1)
        list_Classification = driver.find_element(By.ID,'listboxdiv_classifications').find_elements(By.TAG_NAME,'p')
        for Classif in list_Classification:
            Classif.click()
            time.sleep(1)
            # type of value
            value_t = search_block[5]
            driver.execute_script("arguments[0].style.display = 'block';", value_t)
            time.sleep(1)
            filter_box_5 = value_t.find_element(By.CLASS_NAME,'ddlbFilterBox')
            driver.execute_script("arguments[0].style.display = 'block';", filter_box_5)
            time.sleep(1)
            list_value_t = driver.find_element(By.ID,'listboxdiv_measures').find_elements(By.TAG_NAME,'p')
            for value in list_value_t:
                if value.get_attribute('class') == 'not-selectable':
                    pass
                else:
                    value.click()
                    time.sleep(1)
                    # gas 
                    gas = search_block[6]
                    driver.execute_script("arguments[0].style.display = 'block';", gas)
                    time.sleep(1)
                    filter_box_6 = gas.find_element(By.CLASS_NAME,'ddlbFilterBox')
                    driver.execute_script("arguments[0].style.display = 'block';", filter_box_6)
                    time.sleep(1)
                    list_gas = driver.find_element(By.ID,'listboxdiv_gases').find_elements(By.TAG_NAME,'p')
                    for g in list_gas:
                        g.click()
                        time.sleep(1)
                        t_1 = categ.text
                        t_2 = Classif.text
                        if t_2 == 'Total for category':
                            t_2 = ''
                        t_3 = value.text
                        if t_3 == 'No type of value':
                            t_3 = ''
                        t_4 = g.text
                        if t_4 == 'No gas':
                            t_4 = ''
                        title = tit + ' ' + t_1 + ' ' + t_2 + ' ' + t_3  + ' ' + t_4
                        title = title.replace('.','').replace('~$','').replace('~','').replace(':','').replace('/',', ')
                        if len(title)>180 :
                            title = title[:181]
                        GO = driver.find_element(By.CLASS_NAME,'button-group').find_element(By.CLASS_NAME,'button')
                        GO.click()
                        time.sleep(2)
                        if len(driver.find_elements(By.XPATH,'//*[contains(text(),"Table and graph cannot be displayed due to missing data")]')) > 0:
                            pass
                        else:
                            description = driver.find_element(By.XPATH,'//*[contains(text(),"Query results for")]').text
                            notes = driver.find_element(By.CLASS_NAME,'disclaimer-notes').text
                            # table
                            result = driver.find_element(By.ID,'divToPrint')
                            ta = result.find_element(By.CLASS_NAME,'pvtUi')
                            ro = ta.find_elements(By.TAG_NAME,'tr')
                            part_table = ro[2].find_element(By.CLASS_NAME,'pvtOutput')
                            table = part_table.find_element(By.ID,'tableGrid').find_element(By.ID,'dwhPivot')
                            cells_col = table.find_element(By.TAG_NAME,'tr').find_elements(By.TAG_NAME,'th')
                            cells_unit = table.find_elements(By.TAG_NAME,'tr')[1].find_elements(By.TAG_NAME,'th')
                            column = []
                            for cell_c in cells_col[2:]:
                                column.append(cell_c.text)
                            unt = cells_unit[1].text
                            column.insert(0, 'Party')
                            body_table = table.find_element(By.TAG_NAME,'tbody')
                            rows = body_table.find_elements(By.TAG_NAME,'tr')
                            l = []
                            for tr in rows:
                                cells = tr.find_elements(By.CLASS_NAME,'pvtVal')
                                col_0 = tr.find_element(By.CLASS_NAME,'pvtRowLabel')
                                cells_text = [col_0.text] + [cell.text for cell in cells]
                                l.append(cells_text)
                            unit = [unt] * len(rows)
                            df = pd.DataFrame(l,columns=column) 
                            df.insert(1,'Unit',unit)
                            df.to_excel(f'{base_path}\\{title}.xlsx',index = False)
                        # return
                        show = driver.find_element(By.CLASS_NAME,'button-group').find_elements(By.CLASS_NAME,'button')[1]
                        show.click()
                        deselect_gas = filter_box_6.find_elements(By.CLASS_NAME,"ddlbButton")[1]
                        deselect_gas.click()
                        time.sleep(1)
                    deselect_value_t = filter_box_5.find_elements(By.CLASS_NAME,"ddlbButton")[1]
                    driver.execute_script('arguments[0].scrollIntoView(true);', Classification)
                    deselect_value_t.click()
                    time.sleep(1)
                
            deselect_Classification = filter_box_4.find_elements(By.CLASS_NAME,"ddlbButton")[1]
            driver.execute_script('arguments[0].scrollIntoView(true);', category)
            deselect_Classification.click()
            time.sleep(1)
        deselect_category = filter_box_3.find_elements(By.CLASS_NAME,"ddlbButton")[1]
        driver.execute_script('arguments[0].scrollIntoView(true);', year)
        time.sleep(1)
        deselect_category.click()
        time.sleep(1)



NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.133)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0103A813+48355]
	(No symbol) [0x00FCC4B1]
	(No symbol) [0x00ED5358]
	(No symbol) [0x00EBD293]
	(No symbol) [0x00F1E37B]
	(No symbol) [0x00F2C473]
	(No symbol) [0x00F1A536]
	(No symbol) [0x00EF82DC]
	(No symbol) [0x00EF93DD]
	GetHandleVerifier [0x0129AABD+2539405]
	GetHandleVerifier [0x012DA78F+2800735]
	GetHandleVerifier [0x012D456C+2775612]
	GetHandleVerifier [0x010C51E0+616112]
	(No symbol) [0x00FD5F8C]
	(No symbol) [0x00FD2328]
	(No symbol) [0x00FD240B]
	(No symbol) [0x00FC4FF7]
	BaseThreadInitThunk [0x75A300C9+25]
	RtlGetAppContainerNamedObjectPath [0x77937B4E+286]
	RtlGetAppContainerNamedObjectPath [0x77937B1E+238]


In [ ]:
notes = driver.find_element(By.CLASS_NAME,'disclaimer-notes').text
# table
result = driver.find_element(By.ID,'divToPrint')
ta = result.find_element(By.CLASS_NAME,'pvtUi')
ro = ta.find_elements(By.TAG_NAME,'tr')
part_table = ro[2].find_element(By.CLASS_NAME,'pvtOutput')
table = part_table.find_element(By.ID,'tableGrid').find_element(By.ID,'dwhPivot')
cells_col = table.find_element(By.TAG_NAME,'tr').find_elements(By.TAG_NAME,'th')
cells_unit = table.find_elements(By.TAG_NAME,'tr')[1].find_elements(By.TAG_NAME,'th')
column = []
for cell_c in cells_col[2:]:
    column.append(cell_c.text)
unt = cells_unit[1].text
column.insert(0, 'Party')
body_table = table.find_element(By.TAG_NAME,'tbody')
rows = body_table.find_elements(By.TAG_NAME,'tr')
l = []
for tr in rows:
    cells = tr.find_elements(By.CLASS_NAME,'pvtVal')
    col_0 = tr.find_element(By.CLASS_NAME,'pvtRowLabel')
    cells_text = [col_0.text] + [cell.text for cell in cells]
    l.append(cells_text)
unit = [unt] * len(rows)
df = pd.DataFrame(l,columns=column) 
df.insert(1,'Unit',unit)
# df.to_excel(f'{base_path}\\{title}.xlsx',index = False)

In [18]:
result = driver.find_element(By.ID,'divToPrint')
ta = result.find_element(By.CLASS_NAME,'pvtUi')
ro = ta.find_elements(By.TAG_NAME,'tr')
part_table = ro[2].find_element(By.CLASS_NAME,'pvtOutput')
table = part_table.find_element(By.ID,'tableGrid').find_element(By.ID,'dwhPivot')
thead = table.find_element(By.TAG_NAME,'thead')
# thead.text
columns = []
head = thead.find_element(By.TAG_NAME,'tr').find_elements(By.TAG_NAME,'th')
for th in head[2:]:
    columns.append(th.text)
columns
# for row_h in row_head[:-2]:
#     print(row_h.text)

['2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', 'Cumulative']